In [110]:
""" 
Assessing greenspace accessability within Northern Ireland through an interactive map which calculates the nearest proximity of a settlement to a greenspace trail. 
"""
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely.strtree import STRtree
from shapely.ops import nearest_points

In [111]:
"""
Load settlement, trail, and greenspace shapefiles.
"""
settlements = gpd.read_file("data/Settlements/settlements-2015-above-500-threshold.shp").to_crs(epsg=3857)
trails = gpd.read_file("data/Trails/greenspaceoffroadtrails.shp").to_crs(epsg=3857)
greenspace = gpd.read_file("data/Greenspace/greenspace.shp").to_crs(epsg=3857)


In [112]:
"""
Check the EPSG of each data file.
"""
settlements.crs
trails.crs
greenspace.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [113]:
greenspace.head(5)

,SourceID,GUID,Name,Source,Category,Type,PaidAccess,Area_Ha,Verified,ShowOnMap,ORNI_ID,DataAdded,SiteCreate,geometry
0,None,{68D2DF20-1512-4218-A759-1000732E93B0},Conservation Volunteers NI,LPS and Outscape,Amenity Greenspace,Community Garden,No,0.668234,Approximated,Yes,51,2022-09-21,Pre 2023,"POLYGON Z ((-658305.131 7272616.366 0, -658467..."
1,None,{7B20F220-682E-4566-A4FE-2513CC65ED6E},Lough Shore Park,Antrim and Newtownabbey Borough Council,Parks and Gardens,Public Park,No,6.598284,Approximated,Yes,61,2022-09-21,Pre 2023,"POLYGON Z ((-694569.114 7306677.129 0, -694647..."
2,None,{8A4B8A31-EBA9-4E7A-B8D5-CEA0724D848D},Ardmore Recreation Centre,"Armagh City, Banbridge and Craigavon Borough C...",Amenity Greenspace,Playing Fields,No,2.969549,Approximated,Yes,67,2022-09-21,Pre 2023,"POLYGON Z ((-737723.904 7234648.427 0, -737711..."
3,None,{36F2A040-649C-4EB5-9174-7B25C083F489},Clare Glen - Phase 3 - Link Footpath/River Wal...,"Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,3.396109,Approximated,Yes,69,2022-09-21,Pre 2023,"POLYGON Z ((-713809.418 7236506.822 0, -713812..."
4,None,{2F586E46-B9EE-4D13-83B1-9B254FA3A698},"Folly Glen, Armagh","Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,7.849400,Approximated,Yes,70,2022-09-21,Pre 2023,"MULTIPOLYGON Z (((-738837.244 7234074.082 0, -..."


In [114]:
"""
Filter out invalid geometries to avoid errors during distance calculations,
spatial indexing, and map rendering.
"""
settlements = settlements[settlements.is_valid]
trails = trails[trails.is_valid]
greenspace = greenspace[greenspace.is_valid]

In [115]:
"""
Calculate area (in hectares) for each greenspace polygon and store its centroid.
"""
if 'Area_Ha' not in greenspace.columns:
    greenspace['Area_Ha'] = greenspace.geometry.area / 10_000  # Convert m² to ha

greenspace['centroid'] = greenspace.geometry.centroid

In [116]:
"""
Use a spatial index (STRtree) to efficiently find the nearest trail segment
to each settlement. Record both the connecting line and the distance.
"""
trail_geoms = trails.geometry.tolist()
trail_tree = STRtree(trail_geoms)

connection_lines = []  # List to hold LineStrings from settlements to trails
distances = []         # List to hold Euclidean distances (in meters)

for idx, row in settlements.iterrows():
    point = row.geometry.centroid
    nearest_idx = trail_tree.nearest(point)
    nearest_geom = trail_geoms[nearest_idx]
    nearest_point = nearest_points(point, nearest_geom)[1]

    line = LineString([point, nearest_point])
    distance = point.distance(nearest_point)

    connection_lines.append(line)
    distances.append(distance)

# Create GeoDataFrame of connection lines and distances
connections = gpd.GeoDataFrame(
    {'distance_m': distances},
    geometry=connection_lines,
    crs=settlements.crs
)

In [117]:
"""
Check the EPSG of the connections.gpd.
"""
connections.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [118]:
"""
Reproject all spatial layers to EPSG:4326 for compatibility with Folium.
"""
settlements = settlements.to_crs(epsg=4326)
trails = trails.to_crs(epsg=4326)
greenspace = greenspace.to_crs(epsg=4326)
connections = connections.to_crs(epsg=4326)

In [119]:
"""
Check the EPSG of each data file.
"""
settlements.crs
trails.crs
greenspace.crs
connections.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [120]:
"""
Center the map on the geographic centroid of all settlements and initialize a
Folium web map.
"""
center = settlements.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=8)


In [121]:
"""
Overlay greenspace polygons on the map with popups showing name and area.
"""
for _, row in greenspace.iterrows():
    popup = folium.Popup(
        f"<strong>Name:</strong> {row.get('Name', 'N/A')}<br>"
        f"<strong>Category:</strong> {row.get('Category','N/A')}<br>"
        f"<strong>Area:</strong> {round(row['Area_Ha'], 2)} ha",
        max_width=250
    )
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'darkgreen',
            'fillOpacity': 0.4,
            'weight': 0.5
        },
        popup=popup
    ).add_to(m)


In [122]:
"""
Add trail network lines to the map with a blue style for visual distinction.
"""
folium.GeoJson(
    trails,
    name='Trails',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2
    }
).add_to(m)


In [123]:
"""
Represent each settlement with a black circle marker, using the centroid for placement.
"""
for _, row in settlements.iterrows():
    centroid = row.geometry.centroid
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=4,
        color='black',
        fill=True,
        fill_opacity=0.7,
        tooltip=row.get("Settlement_Name", "Settlement")
    ).add_to(m)

In [124]:
"""
Draw red lines from each settlement to the nearest trail, with tooltip showing distance.
"""
for _, row in connections.iterrows():
    folium.GeoJson(
        row.geometry,
        name='Nearest Trail Line',
        style_function=lambda x: {
            'color': 'red',
            'weight': 1.5
        },
        tooltip=f"Distance: {int(row['distance_m'])} m"
    ).add_to(m)


In [125]:
"""
Add interactive layer control and export the complete map as an HTML file.
"""
folium.LayerControl().add_to(m)
m.save("nearest_trail_access_map.html")

print('nearest_trail_access_map.html')

nearest_trail_access_map.html
